# Capstone PowerBI Audit Log Usage Data

This Databricks notebook is a compilation of the PowerBI audit log ingestion and transformation file and the recommendation engine project. 

Last updated: 12/08/2022

## Part 1: Data collection/ingestion

### Install and import libraries

In [2]:
import json
import pandas as pd
import numpy as np

### Data collection

Pipeline deposits data into hc_datalake: eids_fs storage container in Raw. Data must be read from Raw and analyzed in Refined per organization standards

In [4]:
#raw folder datapath
raw_datapath = "/dbfs/mnt/ca/Raw/CountyAdmin/PBI_usage/"

#refined folder datapath
refined_datapath = "/dbfs/mnt/ca/Refined/CountyAdmin/PBI_usage/"

#empty data container
data = []

#read in json file
for line in open(raw_datapath + 'PowerBI_Log.json', 'r'):
    data.append(json.loads(line))
    
#transform into pandas dataframe
pd_usagedata = pd.DataFrame(data)

#save pandas dataframe back to raw for preservation as csv and to refinsed for transformations
pd_usagedata.to_csv(raw_datapath + 'pbi_usagedata.csv', index = False)
pd_usagedata.to_csv(refined_datapath + 'pbi_usagedata.csv', index = False)

In [ ]:
#checkpoint
pd_usagedata.head()

## Part 2: Exploratory Analysis

In [ ]:
#read in data from refined
usage_data= pd.read_csv(refined_datapath + 'pbi_usagedata.csv')

In [ ]:
useage_data.shape

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(usage_data.WorkSpaceName.value_counts())

In [ ]:
#change the names of the workspaces to be an accurate reflection of workspaces

def workspace(x):
    """ Function to correctly name the workspaces managed by IDA """
    
    if x == 'HC.COVID-19 Response':
        return 'COVID-19 Response'
    
    elif x == 'HC.Analytics':
        return 'Hennepin Analytics'
    
    elif x == 'HC.Labs':
        return 'Hennepin Labs'
    
    elif x == 'HC.DisparityReduction':
        return 'Disparity Reduction'
    
    elif x == 'CIE.Analytics':
        return 'IDA Analytics'
    
    else:
        return 'IDA Analytics'

#apply the function on the workspace
usage_data['ApplicationWorkSpace'] = usage_data['WorkSpaceName'].apply(workspace)

In [ ]:
#remove COVID-19 Workspace
usage_data = usage_data[usage_data.ApplicationWorkSpace != 'COVID-19 Response'] # remove COVID-19 

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(usage_data.WorkSpaceName.value_counts())


In [ ]:
usage_data.shape


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(usage_data.Activity.value_counts())

In [ ]:
usage_data = usage_data[usage_data.Activity == 'ViewReport'] # keep activities of report views

In [ ]:
usage_data.shape

In [ ]:
#Staff file = IDA staff and who has access to HC Analytics

staff_pd = pd.read_csv(raw_datapath + 'staff.csv')
#staff_pd.to_csv(raw_datapath +'staff.csv')
staff_pd.to_csv(refined_datapath +'staff.csv')

staff = pd.read_csv(refined_datapath+'staff.csv')
staff.head(10)

In [ ]:
ida_staff_freq = pd.crosstab(index=staff['IDA_team'], columns='count') 
ida_staff_freq.head()

In [ ]:
#how many decision-makers are part of the Hennepin Analytics access group
hennanalytics_staff_freq = pd.crosstab(index=staff['HennAnalytics'], columns='count') 
hennanalytics_staff_freq.head()

In [ ]:
#Identify IDA staff and those with Hennepin Analytics access
usage_data['IDA'] = usage_data['UserId'].map(staff.set_index('UserId')['IDA_team'])
usage_data['HennAnalytics_access'] = usage_data['UserId'].map(staff.set_index('UserId')['HennAnalytics'])        

#check the set
#usage_data.head()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(usage_data.IDA.value_counts())

In [ ]:
usage_data.info()

In [ ]:
usage_data = usage_data[usage_data.IDA != 'IDA'] # remove IDA staff from views

#check point
#usage_data.head()

In [ ]:
# counting unique values
unique_reports = len(pd.unique(usage_data['ReportName']))

unique_people = len(pd.unique(usage_data['UserKey']))
  
print("Number of unique reports:", unique_reports, "and Number of unique users:", unique_people)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(usage_data.UserKey.value_counts())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  
    print(usage_data.ReportName.value_counts())

In [ ]:
freq_reportnames = pd.crosstab(index=usage_data['ReportName'], columns='count') 
freq_reportnames.sort_values('count', ascending = False).head()

In [ ]:
freq_reportnames.hist(column = 'count')

In [ ]:
ax = freq_reportnames.hist(column='count', bins=30, grid=False, figsize=(8,6), color='#0058a4', zorder=2, rwidth=0.9)

ax = ax[0]
for x in ax:

    # Despine
    x.spines['right'].set_visible(False)
    x.spines['top'].set_visible(False)
    x.spines['left'].set_visible(False)

    # Switch off ticks
    x.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

    # Remove title
    x.set_title("")

    # Set x-axis label
    x.set_xlabel("Number of Report Views", labelpad=20, weight='bold', size=12)

    # Set y-axis label
    x.set_ylabel("Number of Reports", labelpad=20, weight='bold', size=12)

    # Format y-axis label
    x.yaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))

In [ ]:
freq_users = pd.crosstab(index=usage_data['UserId'], columns='count')
ax = freq_users.hist(column='count', bins=30, grid=False, figsize=(8,6), color='#0058a4', zorder=2, rwidth=0.9)

ax = ax[0]
for x in ax:

    # Despine
    x.spines['right'].set_visible(False)
    x.spines['top'].set_visible(False)
    x.spines['left'].set_visible(False)

    # Switch off ticks
    x.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

    # Draw horizontal axis lines
    vals = x.get_yticks()
    for tick in vals:
        x.axhline(y=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

    # Remove title
    x.set_title("")

    # Set x-axis label
    x.set_xlabel("Number of Views", labelpad=20, weight='bold', size=12)

    # Set y-axis label
    x.set_ylabel("Number of Users", labelpad=20, weight='bold', size=12)

    # Format y-axis label
    x.yaxis.set_major_formatter(StrMethodFormatter('{x:,g}'))

In [ ]:
#reduce the dataset to the only contain the features needed

usage_data = usage_data.drop(columns = ['Id', 'DatasetId', 'IsSuccess', 'ItemName', 'ObjectId', 'Operation', 'RecordType', 'ReportId', 'RequestId', 'UserId', 'UserType', 'Workload', 'WorkspaceId', 'UserAgent', 'DataflowType', 'DataConnectivityMode', 'ArtifactId', 'ArtifactName', 'ImportDisplayName', 'ImportId', 'ImportSource', 'ImportType', 'DashboardId', 'DashboardName', 'Datasets', 'Schedules', 'DataflowRefreshScheduleType', 'OrganizationId', 'WorkSpaceName'])

In [ ]:
#this dataset is ready to go to PowerBI flow 

#trusted_datapath = "/dbfs/mnt/ca/Trusted/CountyAdmin/PBI_usage/"

#usage_data.to_csv(trusted_datapath +'pbi_usagedata.csv')

## Part 3: Recommendation Engine

In [ ]:
#if needed
#raw_datapath = "/dbfs/mnt/ca/Raw/CountyAdmin/PBI_usage/"
#refined_datapath = "/dbfs/mnt/ca/Refined/CountyAdmin/PBI_usage/"
#trusted_datapath = "/dbfs/mnt/ca/Trusted/CountyAdmin/PBI_usage/"

#load in the csv from refined to start transormations

#from refined
#usage_data = pd.read_csv(refined_datapath+'pbi_usagedata.csv')

#from trusted
#usage_data = pd.read_csv(trusted_datapath + 'pbi_usagedata.csv')

In [ ]:
usage_data.info()

In [ ]:
def create_view(x):
    if x == 'ViewReport':
        return 1
    else:
        return 0

usage_data['view'] = usage_data['Activity'].apply(create_view)

usage_data.rename(columns = {'ReportName':'Report'}, inplace = True)


#reduce the dataset to the only contain the features needed



rec_data = usage_data[['Report','UserKey','ApplicationWorkSpace', 'HennAnalytics_access', 'view', 'Activity']]

rec_data.head(7)

In [ ]:
rec_data.groupby('Report').agg(
    person = ('UserKey', 'nunique'),
    views = ('view', 'sum')
).sort_values(by = 'views', ascending = False).head(10)

rec_data_items = rec_data.pivot_table(index = 'UserKey', columns = ['Report'], values = 'view').fillna(0)
rec_data_items.head(25)

In [ ]:
def get_recommendations(df, report):
    """Create a list of report recommendations 
        This is using a report-based collaborative filtering algorithm.
    
    Arguments:
        df (dataframe): Pandas dataframe
        report (string): Column name for target item. 
        
    Returns: 
        recommendations (dataframe): Pandas dataframe of report recommendation 
    """
    
    recommendations = df.corrwith(df[report])
    recommendations.dropna(inplace=True)
    recommendations = pd.DataFrame(recommendations, columns=['correlation']).reset_index()
    recommendations = recommendations.sort_values(by='correlation', ascending=False)
    
    return recommendations

In [ ]:
recommendations = get_recommendations(rec_data_items, 'Enterprise Qualtrics')
recommendations.head(6)

In [ ]:
recommendations = get_recommendations(rec_data_items, 'HC SHAPE 2018 County Results')
recommendations.head(6)

In [ ]:
recommendations = get_recommendations(rec_data_items, 'Welcome to Hennepin Analytics')
recommendations.head(6)